In [42]:
import geopandas as gpd
import pandas as pd
from research.connections.db_client import DBClient
pd.set_option('display.max_columns', None)

In [43]:
db_client = DBClient()

In [44]:
sql_query = "SELECT * FROM public.wdpa_wdoecm_subset"

In [45]:
wdpa_subset = db_client.read_sql(sql_query, geom_col='shape')

In [46]:
wdpa_subset.head()

,id,wdpaid,wdpa_pid,pa_def,name,orig_name,desig,desig_eng,desig_type,iucn_cat,int_crit,marine,rep_m_area,gis_m_area,rep_area,gis_area,no_take,no_tk_area,status,status_yr,gov_type,own_type,mang_auth,mang_plan,cons_obj,supp_info,verif,metadataid,sub_loc,parent_iso3,iso3,shape_length,shape_area,shape,release_public,release_licenced,release_restricted
0,7385,555513193.0,555513193,1,Ogna da Pardiala,Ogna da Pardiala,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.001094,0.014290,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000002,"POLYGON ((9.11499 46.76897, 9.11500 46.76899, ...",None,None,None
1,7397,555693551.0,555693551,1,Riederen,Riederen,Waldreservate,Forest Reserves,National,Not Assigned,Not Applicable,0,0.0,0.0,3.131620,3.131512,Not Applicable,0.0,Designated,2018,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000369,"POLYGON ((7.95353 46.76434, 7.95407 46.76421, ...",None,None,None
2,7404,555513336.0,555513336,1,Fotgs,Fotgs,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.002092,0.043694,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000005,"MULTIPOLYGON (((9.57392 46.58587, 9.57393 46.5...",None,None,None
3,7408,555513429.0,555513429,1,Bächer-Hütta,Bächer-Hütta,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.002076,0.032457,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000004,"MULTIPOLYGON (((9.27418 46.65068, 9.27416 46.6...",None,None,None
4,7411,555513461.0,555513461,1,Albanatscha,Albanatscha,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.003949,0.082630,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000010,"MULTIPOLYGON (((9.78925 46.46576, 9.78927 46.4...",None,None,None


# **1. AREA STATISTICS** 
- Exclude proposed areas

In [6]:
wdpa_subset.status.value_counts()

status
Designated    7579
Adopted          8
Inscribed        4
Name: count, dtype: int64

There are no WPDA `proposed` areas, so every value should be taking in account


The dataset has points and polygons. We should check this and transform the points into polygons.

In [7]:
query= """select distinct st_geometrytype(shape)
from public.wdpa_wdoecm_subset;"""

In [8]:
geom_types = db_client.read_sql(query)

In [9]:
geom_types

,st_geometrytype
0,ST_MultiPoint
1,ST_Polygon
2,ST_MultiPolygon


- Get only REP_AREA <> 0

As the ```WDPA_WDOECM_Manual```indicates on **5.5.2 Known issues Point Data** says 

>"If the area of a point feature has not been reported, it may be best to exclude it. To do this, users should remove points where the
```REP_AREA``` is zero. The remaining points can be buffered by calculating the radius of a circle
proportional to the reported area of the site using GIS tools.


In [49]:
rep_area_0 = """ SELECT t.name,t.rep_area,iso3,st_geometrytype(t.shape) FROM public.wdpa_wdoecm_subset as t
WHERE rep_area = 0
ORDER BY id ASC"""

In [50]:
excluded_ones = db_client.read_sql(rep_area_0)
excluded_ones

,name,rep_area,iso3,st_geometrytype
0,Vedat de caça d'Enclar,0.0,AND,ST_Polygon
1,Vedat de caça de Ransol,0.0,AND,ST_Polygon
2,Vedat de caça de Xixerella,0.0,AND,ST_Polygon
3,Vedat de caça del Parc Natural de la Vall de S...,0.0,AND,ST_Polygon
4,Archipel Des Embiez - Six Fours,0.0,FRA,ST_Polygon
5,Monumento naturale regionale del Sasso Cavalla...,0.0,ITA,ST_Polygon
6,Monumento naturale Funghi di Terra di Rezzago,0.0,ITA,ST_Polygon
7,Monumento naturale Caruga del Torrente Rabbiosa,0.0,ITA,ST_Polygon
8,Monumento naturale regionale di Preia Buia,0.0,ITA,ST_Polygon
9,Monumento naturale regionale del Sasso di Preguda,0.0,ITA,ST_Polygon


## Load Base Layer

In [55]:
base_layer = gpd.read_file('../data/base_layer_subset/base_layer_subset.shp')

In [56]:
base_layer.columns = base_layer.columns.str.lower()

In [58]:
base_layer.to_postgis('base_layer_subset',con=db_client.engine)

In [59]:
query_base = "SELECT * FROM public.base_layer_subset"
base_layer = db_client.read_sql(query_base, geom_col='geometry')

In [60]:
base_layer.head()

,objectid,iso3cd,romnam,adm1nm,pol_type,type,shape_leng,shape_area,map_label,map_color,status,id,geometry
0,5.0,LIE,Liechtenstein,Liechtenstein,Land,Land,0.781427,0.018972,Liechtenstein,LIE,Member State,1,"POLYGON ((9.55550 47.22291, 9.55847 47.22388, ..."
1,36.0,GGY,Guernsey,None,200NM,EEZ,5.974343,0.812747,Guernsey (UK),GBR,Territory,1,"POLYGON ((-2.08791 49.42250, -2.08797 49.42243..."
2,43.0,GBR,United Kingdom of Great Britain & Northern Ire...,None,200NM,EEZ,350.648409,107.718373,United Kingdom of Great Britain & Northern Ire...,GBR,Member State,1,"POLYGON ((-0.48182 50.13153, -0.50136 50.12381..."
3,44.0,HRV,Croatia,None,200NM,EEZ,72.472037,6.165992,Croatia,HRV,Member State,1,"MULTIPOLYGON (((14.08333 43.95833, 14.06174 43..."
4,47.0,JEY,Jersey,None,200NM,EEZ,3.210346,0.280519,Jersey (UK),GBR,Territory,1,"POLYGON ((-2.55723 49.21585, -2.54154 49.22815..."
